In [12]:
from homework_code.dataset.quora_dataset import QuoraDataset, collate
from homework_code.dataset.utils import convert_data_to_tuples
from homework_code.dataset.language import Language
from homework_code.model.network import EmbeddingLSTMNet, SiameseNetwork
from homework_code.model.model_trainer import ModelTrainer
from homework_code.model.utils import create_pretrained_weights
import os
import pandas as pd
import numpy as np
import json


import torch
import torch.nn as nn

from torch.utils.data import DataLoader, SubsetRandomSampler

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Global varibles

In [2]:
# Root and directory paths 
ROOT_PATH = os.getcwd()
DATA_FOLDER_PATH = ROOT_PATH + '/data'
GOOGLE_EMBEDDING_FOLDER_PATH = DATA_FOLDER_PATH + '/google embedding'
GOOGLE_EMBEDDING = GOOGLE_EMBEDDING_FOLDER_PATH + '/GoogleNews-vectors-negative300.bin'

# important paths 
DATASET_FILE_PATH = DATA_FOLDER_PATH + '/mini_quora_dataset_30_50_50k.csv'
EMBEDDING_PATH = GOOGLE_EMBEDDING_FOLDER_PATH + GOOGLE_EMBEDDING

# general variables
EMBEDDING_DIMENSION = 300
EMBEDDING_REQUIRES_GRAD = False
HIDDEN_CELLS = 50
NUM_LAYERS = 1

In [3]:
hparams = {
    'threshold': torch.Tensor([0.5]),
    'learning_rate': 1e-03,
    'epoch': 50,
    'batch_size': 32,
    'hidden_dim': 100,
    'embedding_dim': 300,
    'dropout': 0.3,
}

#### Load File

In [4]:
df = pd.read_csv(DATASET_FILE_PATH)

In [5]:
q_pair, labels = convert_data_to_tuples(df)


Question Pairs:  49999


In [6]:
q_pair[0]

('what are some different ways to make money fast',
 'what are fast ways to make money')

In [7]:
language = Language()
for data in [q_pair]:
    for question_pair in data:
        q1 = question_pair[0]
        q2 = question_pair[1]
        language.addSentence(q1)
        language.addSentence(q2)

In [8]:
quora_dataset = QuoraDataset(q_pair, language.word2index, labels)

In [9]:
for i in quora_dataset:
    print(i)
    break

{'q1': 'what are some different ways to make money fast', 'q2': 'what are fast ways to make money', 'q1_token': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'q2_token': [1, 2, 9, 5, 6, 7, 8], 'labels': 1}


In [10]:
val_split = 0.2
test_split = 0.2
dataset_size = len(quora_dataset)
indices = list(range(dataset_size))
split = int(np.floor(val_split * dataset_size))
shuffle_dataset = True
random_seed = 46

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)
train_dataloader = torch.utils.data.DataLoader(quora_dataset, batch_size=hparams, sampler=train_sampler, collate_fn=collate)
val_dataloader = torch.utils.data.DataLoader(quora_dataset, batch_size=32, sampler=validation_sampler, collate_fn=collate)

print ('Training Set Size {}, Validation Set Size {}'.format(len(train_indices), len(val_indices)))

Training Set Size 40000, Validation Set Size 9999


In [10]:
#pretrained_weights = create_pretrained_weights(GOOGLE_EMBEDDING, EMBEDDING_DIMENSION, language)


In [11]:
#torch.save(pretrained_weights, 'pretrained_weights.pt')

In [12]:
pre_weights = torch.load('pretrained_weights.pt')

In [13]:
embedding_net = EmbeddingLSTMNet(
    EMBEDDING_DIMENSION,
    HIDDEN_CELLS,
    NUM_LAYERS,
    EMBEDDING_REQUIRES_GRAD,
    pre_weights
)


In [15]:
for i, (q1_batch, q1_batch_lengths, q2_batch, q2_batch_lengths, labels) in enumerate(train_dataloader):
    batch1_q1 = (q1_batch)
    length1_q1 = (q1_batch_lengths)
    batch1_q2 = (q2_batch)
    length1_q2 = (q2_batch_lengths)
    break
    

In [16]:
out1 = embedding_net(batch1_q1, length1_q1)
out2 = embedding_net(batch1_q2, length1_q2)

In [17]:
model = SiameseNetwork(embedding_net)

In [18]:
model.forward(batch1_q1, batch1_q2, length1_q1, length1_q2)

tensor([7.3511e-02, 5.4434e-01, 1.7509e-02, 1.0015e-02, 8.0834e-02, 3.5158e-02,
        8.9232e-03, 2.2135e-02, 4.7297e-02, 7.3486e-03, 3.8614e-03, 1.1756e-01,
        8.8321e-02, 3.6325e-01, 2.7495e-05, 1.1076e-02, 1.3872e-02, 2.7444e-01,
        3.0146e-01, 6.4147e-02, 1.5955e-04, 3.6850e-01, 3.5705e-02, 2.7641e-02,
        1.2518e-02, 5.8598e-02, 2.2012e-01, 3.2851e-02, 4.3712e-01, 1.2516e-02,
        1.9187e-01, 1.3773e-02], grad_fn=<CopySlices>)

In [19]:
#####

In [20]:
hparams = {
    'threshold': torch.Tensor([0.5]),
    'learning_rate': 1e-02,
    'epoch': 50
}

In [21]:
embedding_net2 = EmbeddingLSTMNet(
    EMBEDDING_DIMENSION,
    HIDDEN_CELLS,
    NUM_LAYERS,
    EMBEDDING_REQUIRES_GRAD,
    pre_weights
)

model2 = SiameseNetwork(embedding_net2)

trainer = ModelTrainer(model2, hparams, train_dataloader, val_dataloader)

In [22]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01 )
num_epochs = 50

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

total_step = len(train_dataloader)
# Threshold 0.5. Since similarity score will be a value between 0 and 1, we will consider all question pair with values greater than threshold as Duplicate
threshold = torch.Tensor([0.5]).to(device)

for epoch in range(num_epochs):
    loss_history = []
    model.train(True)
    train_correct_total = 0
    for i, (q1_batch, q1_batch_lengths, q2_batch, q2_batch_lengths, labels) in enumerate(train_dataloader):

        labels = torch.FloatTensor(labels).to(device)
        
        # Clear grads
        optimizer.zero_grad()
        
        # Run the forward pass
        similarity_score = model(q1_batch, q2_batch, q1_batch_lengths, q2_batch_lengths)
        print(similarity_score.size()), print(labels.size())
        predictions = (similarity_score > threshold).float() * 1
        total = labels.size()[0]
        correct = (predictions == labels).sum().item()
        train_correct_total += correct
        
        # Calculate Loss
        loss = criterion(similarity_score, labels)
        
        # Calculate gradients
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            loss_history.append(loss.item())
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, np.mean(loss_history), (correct / total) * 100))
            
    print('Training Loss: {:.4f}, Training Accuracy: {:.4f}'.format(np.mean(loss_history), (train_correct_total / len(train_indices)) * 100))
    
    model.train(False)
    val_correct_total = 0
    with torch.no_grad():
        for i, (q1_batch, q1_batch_lengths, q2_batch, q2_batch_lengths, labels) in enumerate(val_dataloader):

            labels = torch.FloatTensor(labels).to(device)

            similarity_score = model(q1_batch, q2_batch, q1_batch_lengths, q2_batch_lengths)
            predictions = (similarity_score > threshold).float() * 1
            total = labels.size()[0]
            correct = (predictions == labels).sum().item()
            val_correct_total += correct
        
        avg_acc_val =  val_correct_total * 100 / len(val_indices)
        print ('Validation Set Size {}, Correct in Validation {}, Validation Accuracy {:.2f}%'.format(len(val_indices), val_correct_total, avg_acc_val))


[autoreload of homework_code.model.model_trainer failed: Traceback (most recent call last):
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_ins

torch.Size([32])
tensor([0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])
torch.Size([32])
tensor([0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.])
torch.Size([32])
tensor([0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.])
torch.Size([32])
tensor([1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.,
        0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.])
torch.Size([32])
tensor([1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
        1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1.])
torch.Size([32])
tensor([1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
        1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.])
torch.Size([32])
tensor([0.,

torch.Size([32])
tensor([1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.])
torch.Size([32])
tensor([0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.])
torch.Size([32])
tensor([0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.])
torch.Size([32])
tensor([1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.])
torch.Size([32])
tensor([1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
torch.Size([32])
tensor([0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.])
torch.Size([32])
tensor([0.,

tensor([1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
        0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.])
torch.Size([32])
tensor([1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
        0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1.])
torch.Size([32])
tensor([0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.])
torch.Size([32])
tensor([1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.])
torch.Size([32])
tensor([0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1.])
torch.Size([32])
tensor([0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
        1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.])
torch.Size([32])
tensor([0., 0., 1., 0., 1., 

torch.Size([32])
tensor([0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.])
torch.Size([32])
tensor([0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
        1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.])
torch.Size([32])
tensor([1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0.])
torch.Size([32])
tensor([1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0.])
torch.Size([32])
tensor([1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0.])
torch.Size([32])
tensor([1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.])
torch.Size([32])
tensor([1.,

...

tensor([0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.])
torch.Size([32])
tensor([1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])
torch.Size([32])
tensor([1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.])
torch.Size([32])
tensor([0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])
torch.Size([32])
tensor([1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.])
torch.Size([32])
tensor([0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.])
torch.Size([32])
tensor([0., 0., 1., 0., 1., 

torch.Size([32])
tensor([1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0.])
torch.Size([32])
tensor([1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.])
torch.Size([32])
tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.])
torch.Size([32])
tensor([0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1.])
torch.Size([32])
tensor([1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.])
torch.Size([32])
tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.])
torch.Size([32])
tensor([0.,

torch.Size([32])
tensor([0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.])
torch.Size([32])
tensor([0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.])
torch.Size([32])
tensor([0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0.])
torch.Size([32])
tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1.])
torch.Size([32])
tensor([0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
        0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.])
torch.Size([32])
tensor([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.])
torch.Size([32])
tensor([1.,

torch.Size([32])
tensor([0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1.])
torch.Size([32])
tensor([1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.])
torch.Size([32])
tensor([1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.])
torch.Size([32])
tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1.])
torch.Size([32])
tensor([1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.])
torch.Size([32])
tensor([1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1.])
torch.Size([32])
tensor([1.,

KeyboardInterrupt: 

In [38]:
len(val_indices)

9999

In [39]:
len(val_dataloader)

313

In [13]:
import wandb

In [16]:
import wandb

wandb.login()

/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
wandb: Currently logged in as: maxifor (use `wandb login --relogin` to force relogin)


True

In [17]:
api = wandb.Api()

In [18]:
worker, project, run_id = "maxifor", "IBM-Praktikum Homework 2", "1ethyd0p"
run = api.run(f"{worker}/{project}/{run_id}")

run.display(height=1080)  # you may need to zoom out to see the whole window!

/Users/Maxi/opt/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


True